<a href="https://colab.research.google.com/github/asungii/quantization-experiments/blob/main/SmoothQuant_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Installations</h1>

From SmoothQuant repo. Because the specified CUDA compiler is 11.3, and the default Colab CUDA version is as of 7/29/24 12.2, we have to change the CUDA version in Colab.

I found info on how to do this from https://medium.com/@ajithkumarv/how-to-modify-cuda-gcc-python-versions-in-colab-584ed4113157 and https://github.com/googlecolab/colabtools/issues/4214#issuecomment-1859155789



In [1]:
# setup is from the mit-han-lab GitHub

# these bits are not necessary for Google Colab
# !conda create -n smoothquant python=3.8
# !conda activate smoothquant
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install transformers==4.36.0 accelerate datasets zstandard

# !python setup.py install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 97.2 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.3.1+cu121
    Uninstalling torch-2.3.1+cu121:
      Successfully uninstalled torch-2.3.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.18.1+cu121
    Uninstalling torchvision-0.18.1+cu121:
      Successfully uninstalled torchvision-0.18.1+cu121
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.3.1+cu121
    Uninstalling torchaudio-2.3.1+cu121:
      Successfully uninstalled torchaudio-2.3.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.18.0 requires torch>=2.3.0, but 

In [2]:
! nvcc --version

# well it's 12.2. that's a bit of an issue

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
!sudo mv cuda-ubuntu2004.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.3.0/local_installers/cuda-repo-ubuntu2004-11-3-local_11.3.0-465.19.01-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu2004-11-3-local_11.3.0-465.19.01-1_amd64.deb
!sudo apt-key add /var/cuda-repo-ubuntu2004-11-3-local/7fa2af80.pub
!sudo apt-get update
!sudo apt-get -y install cuda

!nvcc --version

--2024-07-29 14:07:58--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu2004.pin’

cuda-ubuntu2004.pin 100%[===================>]     190  --.-KB/s    in 0s      

2024-07-29 14:07:58 (5.16 MB/s) - ‘cuda-ubuntu2004.pin’ saved [190/190]

--2024-07-29 14:07:58--  https://developer.download.nvidia.com/compute/cuda/11.3.0/local_installers/cuda-repo-ubuntu2004-11-3-local_11.3.0-465.19.01-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 O

In [13]:
!export CUDA_PATH=/usr/local/cuda-11.3/


In [14]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [16]:
%cd /usr/local/

/usr/local


In [17]:
%pwd

'/usr/local'

In [18]:
%ls

bin/    cuda@     cuda-12.2/  etc/    _gcs_config_ops.so/  lib/    licensing/  sbin/   src/
colab/  cuda-12@  cuda-12.5/  games/  include/             lib64/  man@        share/


In [8]:
! python -c "import torch; print(torch.__version__)"

2.2.1+cu121


In [3]:
! git clone https://github.com/mit-han-lab/smoothquant.git

Cloning into 'smoothquant'...
remote: Enumerating objects: 352, done.
remote: Counting objects: 100% (188/188), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 352 (delta 130), reused 116 (delta 100), pack-reused 164
Receiving objects: 100% (352/352), 6.80 MiB | 26.88 MiB/s, done.
Resolving deltas: 100% (202/202), done.


In [4]:
%cd smoothquant

/content/smoothquant


In [5]:
!pip install -e .

Obtaining file:///content/smoothquant
  Preparing metadata (setup.py) ... done
  Running setup.py develop for smoothquant


In [6]:
%cd ..

/content


In [7]:
! git clone https://github.com/Guangxuan-Xiao/torch-int.git

Cloning into 'torch-int'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 1102 (delta 139), reused 118 (delta 118), pack-reused 929
Receiving objects: 100% (1102/1102), 960.91 KiB | 6.81 MiB/s, done.
Resolving deltas: 100% (643/643), done.


In [8]:
%cd torch-int

/content/torch-int


In [9]:
! git config submodule.submodules/cutlass.url https://github.com/NVIDIA/cutlass.git
! git submodule update --init --recursive

Cloning into '/content/torch-int/submodules/cutlass'...
Submodule path 'submodules/cutlass': checked out 'c975e2ccbb2dbf13024568b37ffa3498ed0b3aed'


In [10]:
! source environment.sh
! bash build_cutlass.sh   # this one needs GPU
! python setup.py install

-- CMake Version: 3.30.1
-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/g++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- The CUDA compiler identification is NVIDIA 12.2.140
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- CUDART: /usr/local/cuda/lib64/libcudart.so
-- CUDA Driver: /usr/local/cuda/lib64/stubs/libcuda.so
-- NVRTC: /usr/local/cuda/lib64/libnvrtc.so
-- Default Install Location: install
-- CUDA Compilation Architectures: 80
-- Enable caching of reference results in conv unit tests
-- Enable rigorous conv problem sizes in conv unit tests
-- Using NVCC flags: -DCUTLASS_TEST_LEVEL=0;-DCUTLASS_TEST_ENABLE_CACHED_RESULTS=1;-DCUTLASS_CONV_U

In [11]:
! python -c "import torch; print(torch.__version__)"

1.12.1+cu102


In [12]:
import smoothquant

In [13]:
from smoothquant.opt import Int8OPTForCausalLM
model = Int8OPTForCausalLM.from_pretrained("mit-han-lab/opt-30b-smoothquant")

ModuleNotFoundError: No module named 'smoothquant.opt'